# 0.0. IMPORTS

In [8]:
import math 
import numpy as np
import pandas as pd
import inflection
import datetime

import seaborn as sns
from scipy                 import stats as ss
from matplotlib            import pyplot as plt
from IPython.core.display  import HTML
from IPython.display       import Image
from sklearn.ensemble      import RandomForestRegressor
from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder
from boruta                import BorutaPy

ModuleNotFoundError: No module named 'inflection'

## 0.1. Helper Functions

In [ ]:
def cramer_v(x,y):
    cm = pd.crosstab( x,y ).values
    n = cm.sum()
    r, k = cm.shape

    chi2 = ss.chi2_contingency( cm )[0]
    chi2corr = max( 0, chi2 - (k-1)*(r-1)/(n-1))
    
    kcorr = k - (k-1)**2/(n-1)
    rcorr = r - (r-1)**2/(n-1)
    
    return np.sqrt( (chi2corr/n) / (min( kcorr-1, rcorr-1 ) ) )


def jupyter_settings():
    %matplotlib inline
    %pylab inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set()
    
jupyter_settings()

## 0.2. Loading data

In [ ]:
df_sales_raw = pd.read_csv( 'data/train.csv', low_memory=False)
df_store_raw = pd.read_csv( 'data/store.csv', low_memory=False)

In [ ]:
#merge

df_raw = pd.merge( df_sales_raw, df_store_raw, how='left', on='Store')

In [ ]:
df_raw.sample()

# 1.0. PASSO 01 - DESCRIÇÃO DOS DADOS

In [9]:
df1 = df_raw.copy()

NameError: name 'df_raw' is not defined

## 1.1. Rename Columns

In [ ]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval']
snakecase = lambda x: inflection.underscore( x )

cols_new = list( map( snakecase, cols_old))

#rename
df1.columns = cols_new

## 1.2. Data Dimensions

In [ ]:
print ( 'Number of Rows: {}' .format(df1.shape[0] ) )
print ( 'Number of Cols: {}' .format(df1.shape[1] ) )

## 1.3. Data Types

In [ ]:
df1['date'] = pd.to_datetime( df1['date'])
df1.dtypes

## 1.4. Check NA

In [ ]:
df1.isna().sum()

## 1.5. Fillout NA

In [ ]:
df1.sample()

In [ ]:
#competition_distance   
df1.competition_distance.fillna(200000., inplace=True)

#competition_open_since_month 
df1.competition_open_since_month.fillna(df1.date.dt.month, inplace=True)

#competition_open_since_year
df1.competition_open_since_year.fillna(df1.date.dt.year, inplace=True)

#promo2_since_week
df1.promo2_since_week.fillna(df1.date.dt.isocalendar().week, inplace=True)

#promo2_since_year
df1.promo2_since_year.fillna(df1.date.dt.year, inplace=True)

# substitui os 0 para '', assim não será necessário um dos if's do is_promo
df1['promo_interval'].replace(0, '', inplace=True)
# mesmo aqui
df1['promo_interval'].fillna('', inplace=True) 

# obtem a abreviatura do mês
df1['month_map'] = df1['date'].dt.strftime("%b") 

# como não tem 0's mais, tudo vai ser string, então podemos remover aqueles if's
df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: x['month_map'] in x['promo_interval'].split( ',' ), axis='columns').astype('int')

## 1.6. Change Types

In [ ]:
df1.dtypes

In [ ]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype( 'int64' )
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype( 'int64' )

df1['promo2_since_week'] = df1['promo2_since_week'].astype( 'int64' )
df1['promo2_since_year'] = df1['promo2_since_year'].astype( 'int64' )

df1['is_promo'] = df1['is_promo'].astype( 'int64' )

## 1.7. Descriptive Statistical

In [ ]:
num_attributes = df1.select_dtypes( include=['int64', 'float64'])
cat_attributes = df1.select_dtypes( exclude=['int64', 'float64', 'datetime64[ns]'])

### 1.7.1 Numerical Attributes

In [ ]:
# Central Tendency - mean, median
ct1 = pd.DataFrame( num_attributes.apply( np.mean )).T
ct2 = pd.DataFrame( num_attributes.apply( np.median )).T

# Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std )).T
d2 = pd.DataFrame( num_attributes.apply( min )).T
d3 = pd.DataFrame( num_attributes.apply( max )).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() )).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() )).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() )).T

#concatenate
m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6] ).T.reset_index()
m.columns = ['attribute', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m.head(5)

### 1.7.2 Categorical Attributes

In [ ]:
cat_attributes.apply( lambda x: x.unique().shape[0])

In [ ]:
aux1 = df1[(df1['state_holiday'] != '0' ) & (df1['sales'] > 0)] 

plt.subplot(1,3,1)
sns.boxplot( x='state_holiday', y='sales', data= aux1)

plt.subplot(1,3,2)
sns.boxplot( x='store_type', y='sales', data= aux1)

plt.subplot(1,3,3)
plt.tight_layout()
sns.boxplot( x='assortment', y='sales', data= aux1)

# 2.0. PASSO 02 - FEATURE ENGINEERING

In [ ]:
df2 = df1.copy()

## 2.1. Mapa Mental de Hipóteses

In [ ]:
Image('img\MindMapHypothesis.png')

## 2.2. Criacao das Hipoteses


### 2.2.1 Hipoteses Loja

**1.** Lojas com maior quadro de funcionarios deveriam vender.

**2.** Lojas com maior estoque deveriam vender mais.

**3.** Lojas com maior porte deveriam vender mais. 

**4.** Lojas com menor porte deveriam vender menos. 

**5.** Lojas com maior sortimento deveriam vender mais.

### 2.2.2 Hipoteses Produto

**1.** Lojas que investem mais em Marketing deveriam vender mais.

**2.** Lojas que expoe mais os produtos nas vitrines deveriam vender mais.

**3.** Lojas que tem precos menores nos produtos deveriam vender mais.

**4.** Lojas que tem precos menores por mais tempo deveriam vender mais.

### 2.2.3 Hipoteses tempo

**1.** Lojas que que tem mais feriados deveriam vender menos.

**2.** Lojas que abrem nos primeiros 6 meses deveriam vender mais

**3.** Lojas que abrem nos finais de semana deveriam vender mais.

## 2.3. Lista Final de Hipóteses

### Prioridade serão as hipósteses que já possuem dados para serem validadas

**1.** Lojas com maior sortimentos deveriam vender mais.

**2.** Lojas com mais dias de promoção deveriam vender mais.

**3.** Lojas com competidores à mais tempo deveriam vender mais.

**4.** Lojas com promoções ativas por mais tempo deveriam vender mais.

**5.** Lojas com mais dias de promoção deveriam vender mais.

**6.** Lojas com mais promoções consecutivas deveriam vender mais.

**7.** Lojas abertas durante o feriado de natal deveriam vender mais.

**8.** Lojas deveriam vender mais ao longo dos anos.

**9.** Lojas deveriam vender mais no segundo semestre do ano.

**10.** Lojas deveriam vender mais depois do dia 10 de cada mês.

**11.** Lojas deveriam vender menos aos finais de semana.

**12.** Lojas deveriam vender menos durante feriados escolares.

## 2.4. Feature Engineering

In [ ]:
# year
df2['year'] = df2['date'].dt.year

# month
df2['month'] = df2['date'].dt.month

# day
df2['day'] = df2['date'].dt.day

# week of year
df2['week_of_year'] = df2['date'].dt.isocalendar().week

# year week
df2['year_week'] = df2['date'].dt.strftime( '%Y-%W' )

# competition since
df2[ 'competition_since'] = df2.apply( lambda x: datetime.datetime( year=x['competition_open_since_year'], month=x['competition_open_since_month'], day=1), axis=1)
df2['competition_time_month'] = ((df2['date'] - df2['competition_since'])/30).apply(lambda x: x.days).astype(int)
# promo since
df2['promo_since'] = df2['promo2_since_year'].astype(str) + '-' + df2['promo2_since_week'].astype(str)
df2['promo_since'] = df2['promo_since'].apply( lambda x: datetime.datetime.strptime( x + '-1', '%Y-%W-%w')- datetime.timedelta( days=7))
df2['promo_time_week'] = ((df2['date'] - df2['promo_since'])/7).apply(lambda x: x.days).astype(int)

# assortment
df2['assortment'] = df2['assortment'].apply( lambda  x: 'basic' if x == 'a' else 'extra' if x == 'b' else 'extended')
# state holiday
df2['state_holiday'] = df2['state_holiday'].apply( lambda  x: 'public_holiday' if x == 'a' else 'easter_holiday' if x == 'b' else 'christmas' if x == 'c' else 'regular_day')

In [ ]:
cat_attributes.apply( lambda x: x.unique().shape[0])

In [ ]:
df2.head(4)

In [ ]:
df2.sample(10)

# 3.0. PASSO 03 - FILTRAGEM DE VARIÁVEIS

In [ ]:
df3 = df2.copy()

In [ ]:
df3.head()

In [10]:
df3.sample(10)

NameError: name 'df3' is not defined

## 3.1. Filtragem das Linhas

In [ ]:
df3 = df3[(df3['open'] != 0) & (df3['sales'] > 0)]

## 3.2. Filtragem das Colunas

In [ ]:
cols_drop = ['customers', 'open', 'promo_interval', 'month_map']
df3 = df3.drop( cols_drop, axis=1 )

In [ ]:
df3.sample(10)

# 4.0. PASSO 04 - ANALISE EXPLORATORIA DOS DADOS

In [ ]:
df4 = df3.copy()

## 4.1. Analise Univariada

### 4.1.1. Response Variable

In [ ]:
sns.kdeplot( data=df4, x='sales')

### 4.1.2. Numerical Variable


In [ ]:
num_attributes.hist( bins=25);

### 4.1.3. Categorical Variable

In [ ]:
cat_attributes.head()

In [ ]:
df4['state_holiday'].unique()

In [ ]:
df4['state_holiday'].drop_duplicates()

In [ ]:
df4['store_type'].drop_duplicates()

In [ ]:
df4['assortment'].drop_duplicates()

In [ ]:
# state_holiday
plt.subplot(3,2,1)
dfa = df4[df4['state_holiday'] != 'regular_day']
sns.countplot( data= dfa, x=dfa['state_holiday'])

plt.subplot(3,2,2)
sns.kdeplot( df4[df4['state_holiday'] == 'public_holiday']['sales'], label='public_holiday', shade=True )
sns.kdeplot( df4[df4['state_holiday'] == 'easter_holiday']['sales'], label='easter_holiday', shade=True )
sns.kdeplot( df4[df4['state_holiday'] == 'christmas']['sales'], label='christmas', shade=True )

# store_type
plt.subplot(3,2,3)
sns.countplot( data = df4, x=df4['store_type'])
                 
plt.subplot(3,2,4)
sns.kdeplot( df4[df4['store_type'] == 'a']['sales'], label='a', shade=True )
sns.kdeplot( df4[df4['store_type'] == 'b']['sales'], label='b', shade=True )
sns.kdeplot( df4[df4['store_type'] == 'c']['sales'], label='c', shade=True )
sns.kdeplot( df4[df4['store_type'] == 'd']['sales'], label='d', shade=True )
                
# assortment
plt.subplot(3,2,5)
sns.countplot( data = df4, x=df4['assortment'])
                 
plt.subplot(3,2,6)
sns.kdeplot( df4[df4['assortment'] == 'basic']['sales'], label='basic', shade=True )
sns.kdeplot( df4[df4['assortment'] == 'extended']['sales'], label='extended', shade=True )
sns.kdeplot( df4[df4['assortment'] == 'extra']['sales'], label='extra', shade=True )

                 
                 
                 

## 4.2. Analise Bivariada

### 4.2.1. Resumo das Hipoteses

In [ ]:
tab =[['Hipoteses', 'Conclusão', 'Relevancia'],
     ['H1', 'Falsa','Baixa'],
     ['H2', 'Falsa','Media'],
     ['H3', 'Falsa','Media'],      
     ['H4', 'Falsa','Baixa'],      
     ['H5', '-','-'],      
     ['H6', 'Falsa','Baixa'],      
     ['H7', 'Falsa','Media'],      
     ['H8', 'Falsa','Alta'],      
     ['H9', 'Falsa','Alta'],      
     ['H10', 'Verdadeira','Alta'],      
     ['H11', 'Verdadeira','Alta'],      
     ['H12', 'Verdadeira','Baixa'],
     ]
print( tabulate(tab, headers='firstrow'))

### H1. Lojas com maior sortimentos deveriam vender mais.
**FALSA** Lojas com maior sortimento vendem menos.
**Gera Insight** por ser contra intuitivo

In [ ]:
aux1 =df4[['assortment', 'sales']].groupby( 'assortment').sum().reset_index()
sns.barplot( x='assortment', y='sales', data=aux1)

#### Insight - Assumindo que o Assortment Extra é oque possui maior sortimento, assumimos que a hipótese é falsa, tendo em vista que pelos dados apresentado, é oque possui menor quantidade de vendas.

In [ ]:
aux2 = df4[['year_week','assortment', 'sales']].groupby( ['year_week','assortment']).sum().reset_index()
aux2.pivot( index='year_week', columns='assortment',values='sales').plot()

#### Basic e Extended, possuem o mesmo comportamento ao longo do tempo, o Extra fica muito pequena nessa análise, necessário ver se comportamento sozinha

In [ ]:
aux3 = aux2[aux2['assortment'] == 'extra']
aux3.pivot( index='year_week', columns='assortment',values='sales').plot()

#### Interessante ver como que com uma escola diferente se observa uma variancia maior de vendas ao longo do tempo.

### H2. Lojas com mais dias de promoção deveriam vender mais.
**FALSA** Lojas com compedidores mais próximos vendem mais.**Gera Insight** por ser contra intuitivo

In [ ]:
aux4 = df4[['competition_distance', 'sales']].groupby( 'competition_distance').sum().reset_index()

plt.subplot(1,3,1)
sns.scatterplot( x ='competition_distance', y='sales', data=aux4);

plt.subplot(1,3,2)
bins = list( np.arange(0, 20000, 1000))
aux4['competition_distance_binned'] = pd.cut( aux4['competition_distance'],bins=bins)
aux5 = aux4[['competition_distance_binned','sales']].groupby('competition_distance_binned').sum().reset_index()
sns.barplot( x='competition_distance_binned', y='sales', data=aux5);
plt.xticks(rotation=90);

plt.subplot(1,3,3)
sns.heatmap( aux4.corr( method='pearson'), annot=True);


#### Insight - Possível verificar que competidores mais próximos as lojas vendem mais do que com a distância maior.

### H3. Lojas com competidores à mais tempo deveriam vender mais.
**FALSO** quanto mais recente a competição, maior é a quantidade de vendas.

In [ ]:
aux6 = df4[['competition_open_since_month', 'sales']].groupby( 'competition_open_since_month').sum().reset_index()
sns.barplot( x = 'competition_open_since_month', y ='sales', data=aux6);

#### Necessário realizar uma correlação melhor entre o inicio da competição é a quantidade de vendas.

In [ ]:
plt.subplot( 1,3,1)
aux7 = df4[['competition_time_month', 'sales']].groupby( 'competition_time_month').sum().reset_index()
aux8 = aux7[(aux7['competition_time_month'] < 120 ) & (aux7['competition_time_month'] != 0)]
sns.barplot( x = 'competition_time_month', y ='sales', data=aux8);
plt.xticks( rotation= 90);

plt.subplot( 1,3,2)
sns.regplot( x = 'competition_time_month', y ='sales', data=aux8);

plt.subplot( 1,3,3)
sns.heatmap( aux7.corr( method='pearson'), annot=True);

### H4. Lojas com promoções ativas por mais tempo deveriam vender mais.
**FALSO** Lojas com promoções ativas por mais tempo vendem menos, depois de um certo periodo de promoção.

In [ ]:
aux1 = df4[['promo_time_week', 'sales']].groupby('promo_time_week').sum().reset_index()
sns.barplot( x='promo_time_week', y='sales', data=aux1);

#### Como há uma grande quantidade de dados, é necessário realizar um recorte. Feito para que seja possível visualizar os dados.

In [ ]:
grid = GridSpec(2,3)
plt.subplot(grid[0,0])
aux2 = aux1[aux1['promo_time_week'] > 0] #promo extendido
sns.barplot( x ='promo_time_week', y='sales', data=aux2);
plt.xticks(rotation=90);

plt.subplot(grid[0,1])
sns.regplot( x ='promo_time_week', y='sales', data=aux2);

plt.subplot(grid[1,0])
aux3 = aux1[aux1['promo_time_week'] < 0] #promo regular
sns.barplot( x ='promo_time_week', y='sales', data=aux3);
plt.xticks(rotation=90);

plt.subplot(grid[1,1])
sns.regplot( x ='promo_time_week', y='sales', data=aux3);

plt.subplot(grid[:,2])
sns.heatmap( aux1.corr(method='pearson'),annot=True);


### <s>H5. Lojas com mais dias de promoção deveriam vender mais.</s>

### H6. Lojas com mais promoções consecutivas deveriam vender mais.
**FALSA** Lojas com mais promoções concecutivas vendem menos.

**SERA VALIDADA NO PRÓXIMO CICLO DO CRISP**

In [ ]:
df4[['promo', 'promo2', 'sales']].groupby(['promo', 'promo2']).sum().reset_index()

In [ ]:
aux1 = df4[(df4['promo'] == 1) & (df4['promo2'] == 1)][['year_week', 'sales']].groupby('year_week').sum().reset_index()
ax = aux1.plot()

aux2 = df4[(df4['promo'] == 1) & (df4['promo2'] == 0)][['year_week', 'sales']].groupby('year_week').sum().reset_index()
aux2.plot( ax= ax)

ax.legend( labels=['Tradicional & Extendida', 'Extendida']);

#### Quanto mais recente, próximo de 0, maior é a quantidade de vendas.

### H7. Lojas abertas durante o feriado de natal deveriam vender mais.
**FALSO** Lojas abertas no natal vendem menos

In [11]:
aux = df4[df4['state_holiday'] != 'regular_day']
aux1 = aux[['state_holiday', 'sales']].groupby('state_holiday').sum().reset_index()
sns.barplot(x = 'state_holiday', y='sales', data=aux1);

NameError: name 'df4' is not defined

#### Nota-se que no natal se vendeu menos que o restante das datas. Para verificar se isso ocorreu em todos os anos vou dividir os dados por ano 

In [ ]:
aux2 = aux[['year','state_holiday', 'sales']].groupby(['year','state_holiday']).sum().reset_index()
sns.barplot(x = 'year', y='sales',hue = 'state_holiday', data=aux2);


### H8. Lojas deveriam vender mais ao longo dos anos.
**FALSO** Vendem Menos 

In [ ]:
aux1 = df4[['year','sales']].groupby('year').sum().reset_index()

plt.subplot(1,3,1)
sns.barplot(x='year', y='sales', data=aux1);

plt.subplot(1,3,2)
sns.regplot(x='year', y='sales', data=aux1);

plt.subplot(1,3,3)
sns.heatmap(aux1.corr( method='pearson'),annot=True)

### H9. Lojas deveriam vender mais no segundo semestre do ano.
**FALSO** vendem menos no segundo semestre

In [ ]:
aux1 = df4[['month','sales']].groupby('month').sum().reset_index()

plt.subplot(1,3,1)
sns.barplot(x='month', y='sales', data=aux1);

plt.subplot(1,3,2)
sns.regplot(x='month', y='sales', data=aux1);

plt.subplot(1,3,3)
sns.heatmap(aux1.corr( method='pearson'),annot=True)

### H10. Lojas deveriam vender mais depois do dia 10 de cada mês.
**VERDADEIRA** lojas vendem mais depois do dia 10

In [ ]:
aux1.head()

In [ ]:
aux1 = df4[['day','sales']].groupby('day').sum().reset_index()

plt.subplot(2,2,1)
sns.barplot(x='day', y='sales', data=aux1);

plt.subplot(2,2,2)
sns.regplot(x='day', y='sales', data=aux1);

plt.subplot(2,2,3)
sns.heatmap(aux1.corr( method='pearson'),annot=True)


aux1['before_after'] = aux1['day'].apply(lambda x: 'before_10_days' if x <= 10 else 'after_10_days')
aux2 = aux1[['before_after', 'sales']].groupby('before_after').sum().reset_index()

plt.subplot(2,2,4)
sns.barplot(x='before_after',y='sales',data=aux2);

### H11. Lojas deveriam vender menos aos finais de semana.
**VERDADEIRA** porque lojas vendem menos nos finais de semana

In [ ]:
aux1 = df4[['day_of_week','sales']].groupby('day_of_week').sum().reset_index()

plt.subplot(1,3,1)
sns.barplot(x='day_of_week', y='sales', data=aux1);

plt.subplot(1,3,2)
sns.regplot(x='day_of_week', y='sales', data=aux1);

plt.subplot(1,3,3)
sns.heatmap(aux1.corr( method='pearson'),annot=True)



### H12. Lojas deveriam vender menos durante feriados escolares.
**VERDADEIRA** porque lojas vendem menos durantes os feriados escolares, exceto o mês de Agosto.

In [ ]:
aux1 = df4[['school_holiday','sales']].groupby('school_holiday').sum().reset_index()

plt.subplot(2,1,1)
sns.barplot(x='school_holiday', y='sales', data=aux1);

plt.subplot(2,1,2)
aux2 = df4[['month','school_holiday','sales']].groupby(['month','school_holiday']).sum().reset_index()
sns.barplot(x='month', y='sales',hue='school_holiday', data=aux2); 

In [ ]:
from tabulate import tabulate

## 4.3. Analise Multivariada

### 4.3.1. Numerical Attributes

In [ ]:
correlation = num_attributes.corr( method='pearson')
sns.heatmap( correlation, annot=True);

### 4.3.2. Categorical Attributes

In [ ]:
#somente atributos categoricos
a = df4.select_dtypes( include='object')

#calculando cramer_v
a1 = cramer_v(a['state_holiday'], a['state_holiday'] )
a2 = cramer_v(a['state_holiday'], a['store_type'] )
a3 = cramer_v(a['state_holiday'], a['assortment'] )

a4 = cramer_v(a['store_type'], a['state_holiday'] )
a5 = cramer_v(a['store_type'], a['store_type'] )
a6 = cramer_v(a['store_type'], a['assortment'] )

a7 = cramer_v(a['assortment'], a['state_holiday'] )
a8 = cramer_v(a['assortment'], a['store_type'] )
a9 = cramer_v(a['assortment'], a['assortment'] )

#dataset final
d = pd.DataFrame( {'state_holiday': [a1,a2,a3],
              'store_type': [a4,a5,a6],
              'assortment': [a7,a8,a9]})
d = d.set_index(d.columns)

#mapa de calor 
sns.heatmap(d,annot=True);

# 5.0. PASSO 05 - DATA PREPARATION

In [ ]:
df5 = df4.copy()

## 5.1. Normalizacao

**Como não há nenhuma varíavel numérica com formato de Normal, é aconselhavel não forçar a Normalização e trabalhar com a Reescala no próximo passo**

## 5.2. Rescaling

#### Selecionando atributos númericos

In [ ]:
a = df5.select_dtypes( include=['int64','float64'])

In [ ]:
a.head()

#### Gerando gráficos para detectar outliers

In [ ]:
sns.boxplot(x = 'competition_distance', data = df5)

In [ ]:
sns.boxplot(x = 'competition_time_month', data = df5)

In [ ]:
sns.boxplot(x = 'promo_time_week', data = df5);

In [ ]:
sns.boxplot(x = 'year', data = df5);

#### Normalizando variáveis, respeitando a indicação de outliers para definir o método

In [ ]:
rs = RobustScaler()
mms = MinMaxScaler()

# competition distance
df5['competition_distance'] = rs.fit_transform(df5[['competition_distance']].values)

# competition time month
df5['competition_time_month'] = rs.fit_transform(df5[['competition_time_month']].values)

# promo time week
df5['promo_time_week'] = mms.fit_transform(df5[['promo_time_week']].values)


# year
df5['year'] = mms.fit_transform(df5[['year']].values)



In [ ]:
sns.distplot(df5['competition_distance']);

## 5.3. Transformação

### 5.3.1 Encoding

In [ ]:
# state_holiday (one hot encoding)
df5 = pd.get_dummies(df5, prefix=['state_holiday'], columns=['state_holiday'])

# store_type (label Encoding)
le = LabelEncoder()
df5['store_type'] = le.fit_transform( df5['store_type'])

# assortment (Ordinal Encoding)
assortment_dict = {'basic' : 1,'extra' : 2,'extended' : 3}
df5['assortment'] = df5['assortment'].map( assortment_dict)

### 5.3.2. Response Variable Transformation

In [ ]:
df5['sales'] = np.log1p( df5['sales'])

### 5.3.3 Nature Transformation

In [ ]:
# week of year
df5['week_of_year_sin'] = df5['week_of_year'].apply( lambda x: np.sin(x *( 2. * np.pi/52 ) ) )
df5['week_of_year_cos'] = df5['week_of_year'].apply( lambda x: np.cos(x *( 2. * np.pi/52 ) ) )

# day of week
df5['day_of_week_sin'] = df5['day_of_week'].apply( lambda x: np.sin(x *( 2. * np.pi/7 ) ) )
df5['day_of_week_cos'] = df5['day_of_week'].apply( lambda x: np.cos(x *( 2. * np.pi/7 ) ) )

# month
df5['month_sin'] = df5['month'].apply( lambda x: np.sin(x *( 2. * np.pi/12 ) ) )
df5['month_cos'] = df5['month'].apply( lambda x: np.cos(x *( 2. * np.pi/12 ) ) )
# day
df5['day_sin'] = df5['day'].apply( lambda x: np.cos(x *( 2. * np.pi/30 ) ) )
df5['day_cos'] = df5['day'].apply( lambda x: np.cos(x *( 2. * np.pi/30 ) ) )


# 6.0. PASSO 06 - FEATURE SELECTION

In [ ]:
df6 = df5.copy()

## 6.1.Split dataframe into training an test dataset

In [ ]:
df6.head()

In [12]:
#Necessário retirar váriaveis que foram derivadas.
cols_drop = ['week_of_year', 'day', 'month', 'day_of_week', 'promo_since', 'competition_since', 'year_week']
df6 = df6.drop( cols_drop, axis=1)

NameError: name 'df6' is not defined

In [ ]:
df6[['store', 'date']].groupby( 'store' ).max().reset_index()['date'][0] - datetime.timedelta( days=6*7)

In [ ]:
# necessário separar o DataSet em Treino e Teste

# Para isso utilizaremos a data como determinante para separação

# training dataset
X_train = df6[df6['date'] < '2015-06-19']
y_train = X_train['sales']

# test dataset
X_test = df6[df6['date'] >= '2015-06-19']
y_test = X_test['sales']

print( 'Training Min Date: {}'.format( X_train['date'].min() ) )
print( 'Training Max Date: {}'.format( X_train['date'].max() ) )

print( '\nTest Min Date: {}'.format( X_test['date'].min() ) )
print( 'Test Max Date: {}'.format( X_test['date'].max() ) )


## 6.2.1. Boruta as Feature Selector

In [ ]:
# training and test dataset for Boruta
#X_train_n = X_train.drop( ['date','sales'], axis=1).values
#y_train_n = y_train.values.ravel()

# define RandomForestRegressor
#rf = RandomForestRegressor( n_jobs=-1)

# define Boruta
#boruta = BorutaPy( rf, n_estimators='auto', verbose=2, random_state=42 ).fit( X_train_n, y_train_n)

In [ ]:
#cols_selected = boruta.support_.tolist()

#best features
#X_train_fs = X_train.drop( ['date', 'sales'], axis=1)
#cols_selected_boruta = X_train_fs.iloc[:,cols_selected].columns.to_list()

#not selected boruta
#cols_not_selected_boruta = list( np.setdiff1d( X_train_fs.columns, cols_selected_boruta))

## 6.3. Manual Feature Selection

In [ ]:
cols_selected_boruta = [
    'store',
    'promo',
    'store_type',
    'assortment',
     'competition_distance',
     'competition_open_since_month',
     'competition_open_since_year',
     'promo2',
     'promo2_since_week',
     'promo2_since_year',
     'competition_time_month',
     'promo_time_week',
     'week_of_year_cos',
     'week_of_year_sin',
     'day_of_week_sin',
     'day_of_week_cos',
     'month_cos',
     'month_sin',
     'day_sin',
     'day_cos']

#columns to add
feat_to_add = ['date', 'sales']

# final features
cols_selected_boruta.extend( feat_to_add)

In [ ]:
cols_selected_boruta

# 7.1 ML

In [ ]:
df7 = df6.copy()